In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [4]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
username = spotify_creds['username']
#scope = spotify_creds['scope']
#redirect_uri = spotify_creds['redirect_url']

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [5]:
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [6]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

In [7]:
sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [8]:
def get_saved_library():
    
    df_saved_tracks = pd.DataFrame()
    track_list = ''
    added_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True
    offset_index = 0

    while more_songs:
        songs = sp.current_user_saved_tracks(offset=offset_index)
        for song in songs['items']:
            #join track ids to a string for audio_features function
            track_list += song['track']['id'] +','
            #get the time when the song was added
            added_ts_list.append(song['added_at'])
            #get the title of the song
            title_list.append(song['track']['name'])
            #get popularity
            popularity_list.append(song['track']['popularity'])
            # get album cover
            album_cover_url = song['track']['album']['images'][0]['url']
            #response = requests.get(album_cover_url)
            #img = Image.open(BytesIO(response.content))
            album_cover_list.append(album_cover_url)
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
        track_list = ''
        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
    #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['date_added'] = added_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [9]:
df = get_saved_library()

In [33]:
scope = 'user-top-read'
redirect_uri = 'http://localhost:8885/callback'
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [34]:
sp = spotipy.Spotify(auth=token)

In [35]:
songs = sp.current_user_top_artists()

In [38]:
len(songs['items'])

20

In [45]:
title_list = []

for song in songs['items']:
    title_list.append(song['name'])

In [75]:
def get_top_artists():
    df_top_artists = pd.DataFrame()
    artists_list = []
    genres_list = []
    artists_pic_list = []
    popularity_list = []
    artists = sp.current_user_top_artists()
    
    for artist in artists['items']:
        artists_pic_list.append(artist['images'][0]['url'])
        artists_list.append(artist['name'])
        genres_list.append(artist['genres'][0])
        popularity_list.append(artist['popularity'])
        
    df_top_artists['artist'] = artists_pic_list 
    df_top_artists['name'] = artists_list
    df_top_artists['genere'] = genres_list
    df_top_artists['popularity'] = popularity_list
    
    return df_top_artists

In [76]:
df = get_top_artists()

In [77]:
df

,artist,name,genere,popularity
0,https://i.scdn.co/image/ab6761610000e5eb66d17e...,Martin Garrix,dance pop,81
1,https://i.scdn.co/image/ab6761610000e5eb67828d...,R3HAB,dance pop,80
2,https://i.scdn.co/image/ab6761610000e5eba9989f...,Jay Chou,c-pop,74
3,https://i.scdn.co/image/ab6761610000e5eb8cb651...,Tiësto,big room,87
4,https://i.scdn.co/image/ab6761610000e5eb8ae7f2...,Justin Bieber,canadian pop,96
5,https://i.scdn.co/image/ab6761610000e5eb549aa0...,Afrojack,big room,77
6,https://i.scdn.co/image/ab6761610000e5eb02ccfd...,Dimitri Vegas & Like Mike,belgian dance,74
7,https://i.scdn.co/image/ab6761610000e5ebc02d41...,Alan Walker,electro house,85
8,https://i.scdn.co/image/ab6761610000e5eb6b5fbf...,ILLENIUM,edm,78
9,https://i.scdn.co/image/ab6761610000e5ebf17d4a...,Gryffin,dance pop,75
